# Defect Pixel Analysis Notebook

Load dependencies.

In [ ]:
# Make sure that any changes to the dependencies are automatically updated
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
# Tools for opening image stacks and relevant data from fits headers
from cosmos_helper_funcs import get_stacks, label_plot, cosmos_gain_dict, get_mean_images
# Tools for making defect pixel histograms
from defect_pixel_funcs import *
from scipy.optimize import curve_fit

Analyze bias images. A few things we're looking for in the next few cell blocks are the following: 1) "Bright Bias" oddities, where for some reason bias frames (1 us dark exposures) tend to be brighter than longer exposures. 2) "Even/Odd Row Patterns," where even rows have systematically different bias levels than odd rows.

In [ ]:
# The folder in which bias image stacks are stored.
bias_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Bias Images/HS_HG_RS'
# Load the image stacks. Each element of bias_stacks is a dictionary containing relevant header data and the image stack itself.
bias_stacks = get_stacks(bias_folder)


In [ ]:
# Get an array of the exposure times represented in the stacks
print(bias_stacks[0].keys())

In [ ]:
# For characterizing lazy pixels, using flat field images around 50% saturation.
gray_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Defect Pixels/Gray Images'
stack = get_stacks(gray_folder)
gray_image_HSHGRS = np.median(stack[2]['imagestack'], axis=0).astype(float) - np.median(stack[1]['imagestack'], axis=0).astype(float)
gray_image_HSHGGS = np.median(stack[7]['imagestack'], axis=0).astype(float) - np.median(stack[12]['imagestack'], axis=0).astype(float)
gray_image_HSLGRS = np.median(stack[5]['imagestack'], axis=0).astype(float) - np.median(stack[3]['imagestack'], axis=0).astype(float)
gray_image_HSLGGS = np.median(stack[9]['imagestack'], axis=0).astype(float) - np.median(stack[0]['imagestack'], axis=0).astype(float)
gray_image_LSLGRS = np.median(stack[8]['imagestack'], axis=0).astype(float) - np.median(stack[4]['imagestack'], axis=0).astype(float)
gray_image_LSLGGS = np.median(stack[13]['imagestack'], axis=0).astype(float) - np.median(stack[11]['imagestack'], axis=0).astype(float)
gray_image_HDR = np.median(stack[6]['imagestack'], axis=0).astype(float) - np.median(stack[10]['imagestack'], axis=0).astype(float)

In [ ]:
cutoff_level = 5
defectMapGrayHSHGRS = plot_defect_hist(gray_image_HSHGRS, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[2], bits=14, gain=cosmos_gain_dict['XRgain_HSHGRS'])[0]
defectMapGrayHSHGGS = plot_defect_hist(gray_image_HSHGGS, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[7], bits=14, gain=cosmos_gain_dict['XRgain_HSHGGS'])[0]
defectMapGrayHSLGRS = plot_defect_hist(gray_image_HSLGRS, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[5], bits=14, gain=cosmos_gain_dict['XRgain_HSLGRS'])[0]
defectMapGrayHSLGGS = plot_defect_hist(gray_image_HSLGGS, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[9], bits=14, gain=cosmos_gain_dict['XRgain_HSLGGS'])[0]
defectMapGrayLSLGRS = plot_defect_hist(gray_image_LSLGRS, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[8], bits=16, gain=cosmos_gain_dict['XRgain_LSLGRS'])[0]
defectMapGrayLSLGGS = plot_defect_hist(gray_image_LSLGGS, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[13], bits=16, gain=cosmos_gain_dict['XRgain_LSLGGS'])[0]
defectMapGrayHDR = plot_defect_hist(gray_image_HDR, clip_type='sigma', cutoff_level=cutoff_level, num_bins=2048, filedata=stack[6], bits=18, gain=cosmos_gain_dict['DSgain_HDRRS'])[0]

hdu = fits.PrimaryHDU(defectMapGrayHSHGRS)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayHSHGRS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapGrayHSHGGS)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayHSHGGS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapGrayHSLGRS)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayHSLGRS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapGrayHSLGGS)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayHSLGGS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapGrayLSLGRS)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayLSLGRS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapGrayLSLGGS)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayLSLGGS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapGrayHDR)
hdu.writeto(gray_folder + '/../Defect Pixel Maps/DefectMapGrayHDR.fits', overwrite=True)

In [ ]:
# Find number of NaNs shared by defect maps
fullDefectMapGray = defectMapGrayHSHGRS * defectMapGrayHSHGGS * defectMapGrayHSLGRS * defectMapGrayHSLGGS * defectMapGrayLSLGRS * defectMapGrayLSLGGS * defectMapGrayHDR
plt.imshow(fullDefectMapGray)
plt.show()
print(np.sum(np.isnan(fullDefectMapGray)) / np.size(fullDefectMapGray) * 100)


In [ ]:
# For characterizing hot pixels, using dark images.
dark_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Fall 2024 Data/Dark Frames/120s'
dark_stack = get_stacks(dark_folder)
dark_image_HSHGRS = np.median(dark_stack[0]['imagestack'], axis=0).astype(float)
dark_image_HSHGGS = np.median(dark_stack[1]['imagestack'], axis=0).astype(float)
dark_image_CMS = np.median(dark_stack[2]['imagestack'], axis=0).astype(float)
# dark_image_HSHGRS = np.median(dark_stack[6]['imagestack'], axis=0).astype(float)
# dark_image_HSHGGS = np.median(dark_stack[2]['imagestack'], axis=0).astype(float)
# dark_image_HSLGRS = np.median(dark_stack[4]['imagestack'], axis=0).astype(float)
# dark_image_HSLGGS = np.median(dark_stack[5]['imagestack'], axis=0).astype(float)
# dark_image_LSLGRS = np.median(dark_stack[3]['imagestack'], axis=0).astype(float)
# dark_image_LSLGGS = np.median(dark_stack[0]['imagestack'], axis=0).astype(float)
# dark_image_HDR = np.median(dark_stack[1]['imagestack'], axis=0).astype(float)

In [ ]:
bias_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Fall 2024 Data/Dark Frames/1us'
bias_stack = get_stacks(bias_folder)
bias_image_HSHGRS = np.median(bias_stack[0]['imagestack'], axis=0).astype(float)
# Save median bias frames
# fits.writeto(bias_folder + '/median_bias_1s_HSHGGS.fits', bias_1s_HSHGGS, overwrite=True)
# fits.writeto(bias_folder + '/median_bias_1s_HDR.fits', bias_1s_HDR, overwrite=True)
# fits.writeto(bias_folder + '/median_bias_1s_HSHGRS.fits', bias_1s_HSHGRS, overwrite=True)
# fits.writeto(bias_folder + '/median_bias_1s_HSLGRS.fits', bias_1s_HSLGRS, overwrite=True)
# fits.writeto(bias_folder + '/median_bias_1s_HSLGGS.fits', bias_1s_HSLGGS, overwrite=True)
# fits.writeto(bias_folder + '/median_bias_1s_LSLGRS.fits', bias_1s_LSLGRS, overwrite=True)
# fits.writeto(bias_folder + '/median_bias_1s_LSLGGS.fits', bias_1s_LSLGGS, overwrite=True)

In [ ]:
bias_1s_HSHGRS = fits.getdata(bias_folder + '/median_bias_1s_HSHGRS.fits')
bias_1s_HSHGGS = fits.getdata(bias_folder + '/median_bias_1s_HSHGGS.fits')
bias_1s_HSLGRS = fits.getdata(bias_folder + '/median_bias_1s_HSLGRS.fits')
bias_1s_HSLGGS = fits.getdata(bias_folder + '/median_bias_1s_HSLGGS.fits')
bias_1s_LSLGRS = fits.getdata(bias_folder + '/median_bias_1s_LSLGRS.fits')
bias_1s_LSLGGS = fits.getdata(bias_folder + '/median_bias_1s_LSLGGS.fits')
bias_1s_HDR = fits.getdata(bias_folder + '/median_bias_1s_HDR.fits')

In [ ]:
# plt.imshow(bias_1s_HSHGRS, vmin=50, vmax=300)
# plt.imshow(bias_1us_HSHGRS, vmin=50, vmax=300)
# plt.imshow(bias_1s_HSHGRS - bias_1us_HSHGRS, vmin=-20, vmax=20)
plt.imshow(dark_image_HSHGRS - bias_1s_HSHGRS, vmin=-20, vmax=10)
plt.colorbar()
plt.show()

In [ ]:
clean_dark_HSHGRS_120s = dark_image_HSHGRS - bias_1s_HSHGRS
clean_dark_HSHGGS_120s = dark_image_HSHGGS - bias_1s_HSHGGS
clean_dark_HSLGRS_120s = dark_image_HSLGRS - bias_1s_HSLGRS
clean_dark_HSLGGS_120s = dark_image_HSLGGS - bias_1s_HSLGGS
clean_dark_LSLGRS_120s = dark_image_LSLGRS - bias_1s_LSLGRS
clean_dark_LSLGGS_120s = dark_image_LSLGGS - bias_1s_LSLGGS
clean_dark_HDR_120s = dark_image_HDR - bias_1s_HDR
# clean_dark_HSHGRS_600s = dark_image_HSHGRS_600s - bias_1s_HSHGRS
# clean_dark_HSHGGS_600s = dark_image_HSHGGS_600s - bias_1s_HSHGGS
# clean_dark_HDR_600s = dark_image_HDR_600s - bias_1s_HDR


In [ ]:
%matplotlib inline
cutoff_level = 120
defectMapDarkHSHGRS = plot_defect_hist(clean_dark_HSHGRS_120s, clip_type='absolute', cutoff_level=114, num_bins=4096, filedata=dark_stack[6], bits=14, gain=cosmos_gain_dict['XRgain_HSHGRS'], show_gaussian=False)[0]
defectMapDarkHSHGGS = plot_defect_hist(clean_dark_HSHGGS_120s, clip_type='absolute', cutoff_level=117, num_bins=4096, filedata=dark_stack[2], bits=14, gain=cosmos_gain_dict['XRgain_HSHGGS'], show_gaussian=False)[0]
defectMapDarkHSLGRS = plot_defect_hist(clean_dark_HSLGRS_120s, clip_type='absolute', cutoff_level=14.3, num_bins=4096, filedata=dark_stack[4], bits=14, gain=cosmos_gain_dict['XRgain_HSLGRS'], show_gaussian=False)[0]
defectMapDarkHSLGGS = plot_defect_hist(clean_dark_HSLGGS_120s, clip_type='absolute', cutoff_level=15.3, num_bins=4096, filedata=dark_stack[5], bits=14, gain=cosmos_gain_dict['XRgain_HSLGGS'], show_gaussian=False)[0]
defectMapDarkLSLGRS = plot_defect_hist(clean_dark_LSLGRS_120s, clip_type='absolute', cutoff_level=54.5, num_bins=4096, filedata=dark_stack[3], bits=16, gain=cosmos_gain_dict['XRgain_LSLGRS'], show_gaussian=False)[0]
defectMapDarkLSLGGS = plot_defect_hist(clean_dark_LSLGGS_120s, clip_type='absolute', cutoff_level=59.1, num_bins=4096, filedata=dark_stack[0], bits=16, gain=cosmos_gain_dict['XRgain_LSLGGS'], show_gaussian=False)[0]
defectMapDarkHDR = plot_defect_hist(clean_dark_HDR_120s, clip_type='absolute', cutoff_level=294, num_bins=2048, filedata=dark_stack[1], bits=18, gain=cosmos_gain_dict['DSgain_HDRRS'], show_gaussian=False)[0]

In [ ]:
# Save defect maps
hdu = fits.PrimaryHDU(defectMapDarkHSHGRS)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkHSHGRS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapDarkHSHGGS)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkHSHGGS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapDarkHSLGRS)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkHSLGRS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapDarkHSLGGS)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkHSLGGS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapDarkLSLGRS)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkLSLGRS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapDarkLSLGGS)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkLSLGGS.fits', overwrite=True)
hdu = fits.PrimaryHDU(defectMapDarkHDR)
hdu.writeto(dark_folder + '/../Defect Pixel Maps/DefectMapDarkHDR.fits', overwrite=True)

In [ ]:
fullDefectMapDark = defectMapDarkHSHGRS * defectMapDarkHSHGGS * defectMapDarkHSLGRS * defectMapDarkHSLGGS * defectMapDarkLSLGRS * defectMapDarkLSLGGS * defectMapDarkHDR
print(np.sum(np.isnan(fullDefectMapDark)) / np.size(fullDefectMapDark) * 100)
# fullDefectMapHSHGRS = defectMapGrayHSHGRS * defectMapDarkHSHGRS
# fullDefectMapHSHGGS = defectMapGrayHSHGGS * defectMapDarkHSHGGS
# fullDefectMapHSLGRS = defectMapGrayHSLGRS * defectMapDarkHSLGRS
# fullDefectMapHSLGGS = defectMapGrayHSLGGS * defectMapDarkHSLGGS
# fullDefectMapLSLGRS = defectMapGrayLSLGRS * defectMapDarkLSLGRS
# fullDefectMapLSLGGS = defectMapGrayLSLGGS * defectMapDarkLSLGGS
# fullDefectMapHDR = defectMapGrayHDR * defectMapDarkHDR
# print(np.sum(np.isnan(fullDefectMapHSHGRS)) / np.size(fullDefectMapHSHGRS) * 100)
# print(np.sum(np.isnan(fullDefectMapHSHGGS)) / np.size(fullDefectMapHSHGGS) * 100)
# print(np.sum(np.isnan(fullDefectMapHSLGRS)) / np.size(fullDefectMapHSLGRS) * 100)
# print(np.sum(np.isnan(fullDefectMapHSLGGS)) / np.size(fullDefectMapHSLGGS) * 100)
# print(np.sum(np.isnan(fullDefectMapLSLGRS)) / np.size(fullDefectMapLSLGRS) * 100)
# print(np.sum(np.isnan(fullDefectMapLSLGGS)) / np.size(fullDefectMapLSLGGS) * 100)
# print(np.sum(np.isnan(fullDefectMapHDR)) / np.size(fullDefectMapHDR) * 100)
# fullDefectMap = fullDefectMapGray * fullDefectMapDark
# print(np.sum(np.isnan(fullDefectMap)) / np.size(fullDefectMap) * 100)

In [ ]:
fullDefectMapDark = defectMapDarkHSHGRS * defectMapDarkHSHGGS * defectMapDarkHSLGRS * defectMapDarkHSLGGS * defectMapDarkLSLGRS * defectMapDarkLSLGGS * defectMapDarkHDR
print(np.sum(np.isnan(fullDefectMapDark[:15])) / np.size(fullDefectMapDark[:15]) * 100)

In [ ]:
# # Save median bias frames
# median_bias_HSHGGS = np.median(stack[2]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_HSHGGS.fits', median_bias_HSHGGS, overwrite=True)
# median_bias_HDR = np.median(stack[0]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_HDR.fits', median_bias_HDR, overwrite=True)
# median_bias_HSHGRS = np.median(stack[4]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_HSHGRS.fits', median_bias_HSHGRS, overwrite=True)
# median_bias_HSLGRS = np.median(stack[3]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_HSLGRS.fits', median_bias_HSLGRS, overwrite=True)
# median_bias_HSLGGS = np.median(stack[6]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_HSLGGS.fits', median_bias_HSLGGS, overwrite=True)
# median_bias_LSLGGRS = np.median(stack[1]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_LSLGRS.fits', median_bias_LSLGGRS, overwrite=True)
# median_bias_LSLGGS = np.median(stack[5]['imagestack'], axis=0)
# fits.writeto(User_myFolder + '/../median_bias_LSLGGS.fits', median_bias_LSLGGS, overwrite=True)

In [ ]:
multiplier_list = np.linspace(1, 20, 50)
iter_list_HSHGRS = np.zeros_like(multiplier_list)
iter_list_HSHGGS = np.zeros_like(multiplier_list)
iter_list_HSLGRS = np.zeros_like(multiplier_list)
iter_list_HSLGGS = np.zeros_like(multiplier_list)
iter_list_LSLGRS = np.zeros_like(multiplier_list)
iter_list_LSLGGS = np.zeros_like(multiplier_list)
iter_list_HDR = np.zeros_like(multiplier_list)
for i, sigma_multiplier in enumerate(multiplier_list):
    defectMapDarkHSHGRS, medianDark, stdDark, iter_HSHGRS = sigma_clip_map(medStackHSHGRS, cutoff_level=sigma_multiplier, cut_low=False)
    iter_list_HSHGRS[i] = iter_HSHGRS
    # defectMapDarkHSHGGS, medianDark, stdDark, iter_HSHGGS = defect_pix_map(medStackHSHGGS, cutoff_level=sigma_multiplier)
    # iter_list_HSHGGS[i] = iter_HSHGGS
    # defectMapDarkHSLGRS, medianDark, stdDark, iter_HSLGRS = defect_pix_map(medStackHSLGRS, cutoff_level=sigma_multiplier)
    # iter_list_HSLGRS[i] = iter_HSLGRS
    # defectMapDarkHSLGGS, medianDark, stdDark, iter_HSLGGS = defect_pix_map(medStackHSLGGS, cutoff_level=sigma_multiplier)
    # iter_list_HSLGGS[i] = iter_HSLGGS
    # defectMapDarkLSLGRS, medianDark, stdDark, iter_LSLGRS = defect_pix_map(medStackLSLGRS, cutoff_level=sigma_multiplier)
    # iter_list_LSLGRS[i] = iter_LSLGRS
    # defectMapDarkLSLGGS, medianDark, stdDark, iter_LSLGGS = defect_pix_map(medStackLSLGGS, cutoff_level=sigma_multiplier)
    # iter_list_LSLGGS[i] = iter_LSLGGS
    # defectMapDarkHDR, medianDark, stdDark, iter_HDR = defect_pix_map(medStackHDR, cutoff_level=sigma_multiplier)
    # iter_list_HDR[i] = iter_HDR
    print(sigma_multiplier, iter_HSHGRS)

In [ ]:
plt.plot(multiplier_list, iter_list_HSHGRS, label='HSHGRS')
# plt.plot(multiplier_list, iter_list_HSHGGS, label='HSHGGS')
# plt.plot(multiplier_list, iter_list_HSLGRS, label='HSLGRS')
# plt.plot(multiplier_list, iter_list_HSLGGS, label='HSLGGS')
# plt.plot(multiplier_list, iter_list_LSLGGS, label='LSHGGS')
# plt.plot(multiplier_list, iter_list_LSLGRS, label='LSLGRS')
# plt.plot(multiplier_list, iter_list_HDR, label='HDR')
plt.xlabel('Sigma Multiplier')
plt.ylabel('Number of Sigma-Clipping Iterations to Convergence')
plt.legend()
plt.show()

In [ ]:
cutoff_level = 3
plot_defect_hist(medStackGray, cutoff_level=cutoff_level, num_bins=1024)
defectMapGray, medianGray, stdGray = defect_pix_map(medStackGray, cutoff_level=cutoff_level)
cleanArrayGray = medStackGray * defectMapGray
# count NaNs in cleanArray
nanCount = np.count_nonzero(np.isnan(cleanArrayGray))
print(f'Number of NaNs in cleanArray: {nanCount}')
plt.imshow(cleanArrayGray, cmap='gray')
plt.show()
plt.imshow(defectMapGray, cmap='gray')
plt.show()

In [ ]:
%matplotlib inline
full_defect_map = defectMapDark * defectMapGray
nanCount = np.count_nonzero(np.isnan(full_defect_map))
print(nanCount)
plt.imshow(full_defect_map, cmap='gray')
plt.show()

In [ ]:
multiplier_list = np.linspace(1, 20, 10)
iter_list_HSHGRS = np.zeros_like(multiplier_list)
iter_list_HSHGGS = np.zeros_like(multiplier_list)
iter_list_HSLGRS = np.zeros_like(multiplier_list)
iter_list_HSLGGS = np.zeros_like(multiplier_list)
iter_list_LSLGRS = np.zeros_like(multiplier_list)
iter_list_LSLGGS = np.zeros_like(multiplier_list)
iter_list_HDR = np.zeros_like(multiplier_list)
for i, sigma_multiplier in enumerate(multiplier_list):
    defectMapDarkHSHGRS, medianDark, stdDark, iter_HSHGRS = mad_clip_map(medStackHSHGRS, mad_level=sigma_multiplier, cut_low=False)
    iter_list_HSHGRS[i] = iter_HSHGRS
    defectMapDarkHSHGGS, medianDark, stdDark, iter_HSHGGS = mad_clip_map(medStackHSHGGS, mad_level=sigma_multiplier)
    iter_list_HSHGGS[i] = iter_HSHGGS
    print(i)

In [ ]:
lslgrs_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Defect Pixels/LS_LG_RS'
lslgrs_stack = get_stacks(lslgrs_folder)
mean_img_dict = get_mean_images(lslgrs_stack)

In [ ]:
plt.imshow(mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'], vmin=-30, vmax=0)
label_plot(lslgrs_stack[1])
plt.colorbar()
plt.title('1s Exposure - 1us Exposure')
plt.show()
plt.hist((mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us']).flatten(), bins=100, range=(-50, 20), histtype='step', label='1s - 1us')
plt.hist((mean_img_dict['LS_LG_RS_5s'] - mean_img_dict['LS_LG_RS_1us']).flatten(), bins=100, range=(-50, 20), histtype='step', label='5s - 1us')
plt.hist((mean_img_dict['LS_LG_RS_120s'] - mean_img_dict['LS_LG_RS_1us']).flatten(), bins=100, range=(-50, 20), histtype='step', label='120s - 1us')
plt.hist((mean_img_dict['LS_LG_RS_600s'] - mean_img_dict['LS_LG_RS_1us']).flatten(), bins=100, range=(-50, 20), histtype='step', label='600s - 1us')
label_plot(lslgrs_stack[1])
plt.legend()
plt.xlabel('Difference in Pixel Value')
plt.ylabel('Number of Pixels')
plt.title('Bias-Subtracted Pixel Values')
plt.show()

In [ ]:
%matplotlib qt
col_sum_1us = (mean_img_dict['LS_LG_RS_1us']).mean(axis=1)
col_sum_1s = (mean_img_dict['LS_LG_RS_1s']).mean(axis=1)
col_sum_5s = (mean_img_dict['LS_LG_RS_5s']).mean(axis=1)
col_sum_120s = (mean_img_dict['LS_LG_RS_120s']).mean(axis=1)
col_sum_1s_diff = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us']).mean(axis=1)
col_sum_5s_diff = (mean_img_dict['LS_LG_RS_5s'] - mean_img_dict['LS_LG_RS_1us']).mean(axis=1)
# Take only rows with even indices
col_sum_1us_even = col_sum_1us[::2]
col_sum_1us_odd = col_sum_1us[1::2]
col_sum_1s_even = col_sum_1s[::2]
col_sum_1s_odd = col_sum_1s[1::2]
col_sum_5s_even = col_sum_5s[::2]
col_sum_5s_odd = col_sum_5s[1::2]
col_sum_120s_even = col_sum_120s[::2]
col_sum_120s_odd = col_sum_120s[1::2]
plt.plot(col_sum_1us_even, label='Even Rows, 1us')
plt.plot(col_sum_1us_odd, label='Odd Rows, 1us')
# plt.plot(col_sum_1s_even, label='Even Rows, 1s')
# plt.plot(col_sum_1s_odd, label='Odd Rows, 1s')
# plt.plot(col_sum_5s_even, label='Even Rows, 5s')
# plt.plot(col_sum_5s_odd, label='Odd Rows, 5s')
plt.plot(col_sum_120s_even, label='Even Rows, 120s')
plt.plot(col_sum_120s_odd, label='Odd Rows, 120s')
# plt.plot(col_sum_even, label='Even Rows, 1s - 1us')
# plt.plot(col_sum_odd, label='Odd Rows, 1s - 1us')
# plt.plot(col_sum_5s_even, label='Even Rows, 5s - 1us')
# plt.plot(col_sum_5s_odd, label='Odd Rows, 5s - 1us')
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value')
label_plot(lslgrs_stack[1])
plt.ylim(350, 450)
plt.legend()
plt.show()

In [ ]:
%matplotlib qt
plt.imshow(mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'], vmin=-30, vmax=0)
plt.colorbar()
label_plot(lslgrs_stack[1])
plt.title('Difference Between 5s and 1s Dark Frames')
# Put a box around top left corner
plt.gca().add_patch(plt.Rectangle((0, 0), 2030, 8120, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((2030, 0), 2030, 8120, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((4060, 0), 2030, 8120, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((6090, 0), 2030, 8120, fill=False, edgecolor='red'))
plt.show()

In [ ]:
# plt.imshow(dark_image_1s - dark_image_1us, vmin=-10, vmax=10)
# plt.colorbar()
top_left = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[:1624, :4060]
top_right = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[:1624, 4060:]
bottom_left = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[6496:, :4060]
bottom_right = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[6496:, 4060:]
far_left = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[:, :2030]
far_right = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[:, 6090:]
center_left = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[:, 2030:4060]
center_right = (mean_img_dict['LS_LG_RS_1s'] - mean_img_dict['LS_LG_RS_1us'])[:, 4060:6090]
# plt.hist((dark_image_1s - dark_image_1us).flatten(), bins=200, range=(-20, 20), histtype='step', label='Full Frame')
hist_range = (-30, 30)
# plt.hist(top_left.flatten(), bins=100, range=hist_range, histtype='step', label='Top Left')
# plt.hist(top_right.flatten(), bins=100, range=hist_range, histtype='step', label='Top Right')
# plt.hist(bottom_left.flatten(), bins=100, range=hist_range, histtype='step', label='Bottom Left')
# plt.hist(bottom_right.flatten(), bins=100, range=hist_range, histtype='step', label='Bottom Right')
plt.hist(far_left.flatten(), bins=100, range=hist_range, histtype='step', label='Far Left')
plt.hist(far_right.flatten(), bins=100, range=hist_range, histtype='step', label='Far Right')
plt.hist(center_left.flatten(), bins=100, range=hist_range, histtype='step', label='Center Left')
plt.hist(center_right.flatten(), bins=100, range=hist_range, histtype='step', label='Center Right')
# plt.hist((dark_image_5s - dark_image_1us).flatten(), bins=100, range=(-5, 5), histtype='step', label='5s - 1us')
# plt.hist((dark_image_30s - dark_image_1us).flatten(), bins=100, range=(-5, 5), histtype='step', label='30s - 1us')
plt.xlabel('Pixel Value Difference')
plt.ylabel('Number of Pixels')
# Put dashed vertical line at zero
plt.axvline(0, color='black', linestyle='dashed')
label_plot(lslgrs_stack[1])
plt.legend()
plt.show()

In [ ]:
dark_current_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Defect Pixels/HS_HG_GS'
dark_current_stack = get_stacks(dark_current_folder)

In [ ]:
dark_image_1us = np.mean(dark_current_stack[2]['imagestack'], axis=0).astype(float)
dark_image_1s = np.mean(dark_current_stack[7]['imagestack'], axis=0).astype(float)
dark_image_5s = np.mean(dark_current_stack[0]['imagestack'], axis=0).astype(float)
dark_image_30s = np.mean(dark_current_stack[4]['imagestack'], axis=0).astype(float)
dark_image_120s = np.mean(dark_current_stack[3]['imagestack'], axis=0).astype(float)
dark_image_600s = (dark_current_stack[1]['imagestack'][0].astype(float) + dark_current_stack[5]['imagestack'][0].astype(float) + dark_current_stack[5]['imagestack'][1].astype(float) + dark_current_stack[6]['imagestack'][0].astype(float)) / 4

In [ ]:
plt.imshow(dark_image_30s - dark_image_1s, vmin=-5, vmax=5)
plt.colorbar()
label_plot(dark_current_stack[3])
plt.title('Difference Between 5s and 1s Dark Frames')
# Put a box around top left corner
plt.gca().add_patch(plt.Rectangle((0, 0), 4060, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((4060, 0), 4060, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((0, 6496), 4060, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((4060, 6496), 4060, 1624, fill=False, edgecolor='red'))
plt.show()

In [ ]:
# plt.imshow(dark_image_1s - dark_image_1us, vmin=-10, vmax=10)
# plt.colorbar()
top_left = (dark_image_1s - dark_image_1us)[:1624, :4060]
top_right = (dark_image_1s - dark_image_1us)[:1624, 4060:]
bottom_left = (dark_image_1s - dark_image_1us)[6496:, :4060]
bottom_right = (dark_image_1s - dark_image_1us)[6496:, 4060:]
# plt.hist((dark_image_1s - dark_image_1us).flatten(), bins=200, range=(-20, 20), histtype='step', label='Full Frame')
plt.hist(top_left.flatten(), bins=100, range=(-5, 5), histtype='step', label='Top Left')
plt.hist(top_right.flatten(), bins=100, range=(-5, 5), histtype='step', label='Top Right')
plt.hist(bottom_left.flatten(), bins=100, range=(-5, 5), histtype='step', label='Bottom Left')
plt.hist(bottom_right.flatten(), bins=100, range=(-5, 5), histtype='step', label='Bottom Right')
# plt.hist((dark_image_5s - dark_image_1us).flatten(), bins=100, range=(-5, 5), histtype='step', label='5s - 1us')
# plt.hist((dark_image_30s - dark_image_1us).flatten(), bins=100, range=(-5, 5), histtype='step', label='30s - 1us')
plt.xlabel('Pixel Value Difference')
plt.ylabel('Number of Pixels')
# Put dashed vertical line at zero
plt.axvline(0, color='black', linestyle='dashed')
label_plot(dark_current_stack[3])
plt.legend()
plt.show()

In [ ]:
clean_dark_5s = dark_image_5s - dark_image_1s
clean_dark_30s = dark_image_30s - dark_image_1s
clean_dark_120s = dark_image_120s - dark_image_1s
clean_dark_600s = dark_image_600s - dark_image_1s
dc_frame_5s = clean_dark_5s * cosmos_gain_dict['XRgain_HSHGGS'] / 4
dc_frame_30s = clean_dark_30s * cosmos_gain_dict['XRgain_HSHGGS'] / 29
dc_frame_120s = clean_dark_120s * cosmos_gain_dict['XRgain_HSHGGS'] / 119
dc_frame_600s = clean_dark_600s * cosmos_gain_dict['XRgain_HSHGGS'] / 599

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(1, 3)
fig.set_figwidth(15)
def func(x, a):
    return a * x
popt1, pcov1 = curve_fit(func, dc_frame_5s.flatten(), dc_frame_30s.flatten())
popt2, pcov2 = curve_fit(func, dc_frame_30s.flatten(), dc_frame_120s.flatten())
popt3, pcov3 = curve_fit(func, dc_frame_120s.flatten(), dc_frame_600s.flatten())
x_points = np.linspace(0, 2, 100)
ax[0].plot(x_points, popt1[0] * x_points, color='red')
ax[1].plot(x_points, popt2[0] * x_points, color='red')
ax[2].plot(x_points, popt3[0] * x_points, color='red')
hist1 = ax[0].hist2d(dc_frame_5s[:, :].flatten(), dc_frame_30s[:, :].flatten(), bins=100, range=[[-0.1, 2], [-0.1, 2]], norm='log')
hist2 = ax[1].hist2d(dc_frame_30s[:, :].flatten(), dc_frame_120s[:, :].flatten(), bins=150, range=[[-0.1, 2], [-0.1, 2]], norm='log')
hist3 = ax[2].hist2d(dc_frame_120s[:, :].flatten(), dc_frame_600s[:, :].flatten(), bins=200, range=[[-0.1, 2], [-0.1, 2]], norm='log')
plt.colorbar(hist1[3], ax=ax[0])
plt.colorbar(hist2[3], ax=ax[1])
plt.colorbar(hist3[3], ax=ax[2])
ax[0].set_xlabel('Dark Current (e-/s), 5s exposure time')
ax[0].set_ylabel('Dark Current (e-/s), 30s exposure time')
ax[1].set_xlabel('Dark Current (e-/s), 30s exposure time')
ax[1].set_ylabel('Dark Current (e-/s), 120s exposure time')
ax[2].set_xlabel('Dark Current (e-/s), 120s exposure time')
ax[2].set_ylabel('Dark Current (e-/s), 600s exposure time')
ax[0].text(0.5, 0, 'Best Fit Ratio: ' + format(popt1[0], '4.3f'), transform=ax[0].transAxes)
ax[1].text(0.5, 0, 'Best Fit Ratio: ' + format(popt2[0], '4.3f'), transform=ax[1].transAxes)
ax[2].text(0.5, 0, 'Best Fit Ratio: ' + format(popt3[0], '4.3f'), transform=ax[2].transAxes)
label_plot(dark_current_stack[3])
plt.show()

In [ ]:
bins = np.arange(0.1, 2, 0.05)
for i in range(len(bins)-1):
    bin_center = (bins[i] + bins[i+1]) / 2
    bin_points_600s = dc_frame_600s[(dc_frame_120s > bins[i]) & (dc_frame_120s < bins[i+1])]
    print(bin_center, np.size(bin_points_600s), np.mean(bin_points_600s), np.std(bin_points_600s))
    if abs(bin_center - np.mean(bin_points_600s)) > 3 * np.std(bin_points_600s):
        print('Point at which long and short exposure dark current diverge: ', bin_center)
        break

In [ ]:
%matplotlib inline
dark_defect_map_HSHGGS = plot_defect_hist(clean_dark_120s, clip_type='absolute', cut_low=False, cutoff_level=117, num_bins=2048, filedata=dark_current_stack[6], bits=14, gain=cosmos_gain_dict['XRgain_HSHGGS'], show_gaussian=False)[0]

In [ ]:
# Find total number of nans in dark_defect_map_HSHGRS
num_nans = np.sum(np.isnan(dark_defect_map_HSHGGS[100:8020]))
num_nans_total = np.sum(np.isnan(dark_defect_map_HSHGGS))
print(num_nans, num_nans_total, (num_nans_total - num_nans) / num_nans_total * 100)

In [ ]:
%matplotlib qt
plt.imshow(dark_defect_map_HSHGGS, cmap='gray')
plt.show()
hdu = fits.PrimaryHDU(dark_defect_map_HSHGGS)
hdu.writeto(dark_current_folder + '/../Defect Pixel Maps/DefectMapDarkHSHGGS.fits', overwrite=True)

In [ ]:
dark_defect_map_HSHGGS_binary = dark_defect_map_HSHGGS.copy()
dark_defect_map_HSHGGS_binary[np.isnan(dark_defect_map_HSHGGS)] = 0
dark_defect_map_HSHGGS_binary[dark_defect_map_HSHGGS > 0] = 255
# Save as a binary TIFF file
import tifffile
tifffile.imwrite(dark_current_folder + '/../Defect Pixel Maps/HS_HG_GS_Dark_Outlier_Binary.tif', dark_defect_map_HSHGGS_binary.astype(np.uint8))


In [ ]:
plt.imshow(dark_defect_map_HSHGRS_binary.astype(np.uint8), cmap='gray')
plt.show()

In [ ]:
dark_current_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Dark Current/HS_HG_GS'
dark_current_stack = get_stacks(dark_current_folder)

In [ ]:
dark_frame_1us = np.mean(dark_current_stack[4]['imagestack'], axis=0).astype(float)
dark_frame_1s = np.mean(dark_current_stack[0]['imagestack'], axis=0).astype(float)
dark_frame_5s = np.mean(dark_current_stack[1]['imagestack'], axis=0).astype(float)
dark_frame_30s = np.mean(dark_current_stack[3]['imagestack'], axis=0).astype(float)
dark_frame_600s = np.mean(dark_current_stack[2]['imagestack'], axis=0).astype(float)
dc_frame_1s = (dark_frame_1s - dark_frame_1us) * cosmos_gain_dict['XRgain_HSHGGS']
dc_frame_5s = (dark_frame_5s - dark_frame_1us) * cosmos_gain_dict['XRgain_HSHGGS'] / 5
dc_frame_30s = (dark_frame_30s - dark_frame_1us) * cosmos_gain_dict['XRgain_HSHGGS'] / 30
dc_frame_600s = (dark_frame_600s - dark_frame_1us) * cosmos_gain_dict['XRgain_HSHGGS'] / 600

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(1, 4)
fig.set_figwidth(12)
fig.tight_layout()
ax[0].imshow(dc_frame_1s, vmin=0, vmax=10)
ax[1].imshow(dc_frame_5s, vmin=0, vmax=10)
ax[2].imshow(dc_frame_30s, vmin=0, vmax=10)
ax[3].imshow(dc_frame_600s, vmin=0, vmax=10)
plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 3)
fig.set_figwidth(15)
def func(x, a):
    return a * x
fit_points_1s = dc_frame_1s[(dc_frame_1s < 20) & (dc_frame_5s < 20) & (dc_frame_30s < 20) & (dc_frame_600s < 20)]
fit_points_5s = dc_frame_5s[(dc_frame_1s < 20) & (dc_frame_5s < 20) & (dc_frame_30s < 20) & (dc_frame_600s < 20)]
fit_points_30s = dc_frame_30s[(dc_frame_1s < 20) & (dc_frame_5s < 20) & (dc_frame_30s < 20) & (dc_frame_600s < 20)]
fit_points_600s = dc_frame_600s[(dc_frame_1s < 20) & (dc_frame_5s < 20) & (dc_frame_30s < 20) & (dc_frame_600s < 20)]
popt1, pcov1 = curve_fit(func, fit_points_1s.flatten(), fit_points_5s.flatten())
popt2, pcov2 = curve_fit(func, fit_points_5s.flatten(), fit_points_30s.flatten())
popt3, pcov3 = curve_fit(func, fit_points_30s.flatten(), fit_points_600s.flatten())
x_points = np.linspace(0, 20, 100)
ax[0].plot(x_points, popt1[0] * x_points, color='red')
ax[1].plot(x_points, popt2[0] * x_points, color='red')
ax[2].plot(x_points, popt3[0] * x_points, color='red')
hist1 = ax[0].hist2d(dc_frame_1s[:, :].flatten(), dc_frame_5s[:, :].flatten(), bins=100, range=[[-0.5, 20], [-0.5, 20]], norm='log')
hist2 = ax[1].hist2d(dc_frame_5s[:, :].flatten(), dc_frame_30s[:, :].flatten(), bins=150, range=[[-0.5, 20], [-0.5, 20]], norm='log')
hist3 = ax[2].hist2d(dc_frame_30s[:, :].flatten(), dc_frame_600s[:, :].flatten(), bins=200, range=[[-0.5, 20], [-0.5, 20]], norm='log')
plt.colorbar(hist1[3], ax=ax[0])
plt.colorbar(hist2[3], ax=ax[1])
plt.colorbar(hist3[3], ax=ax[2])
ax[0].set_xlabel('Dark Current (e-/s), 1s exposure time')
ax[0].set_ylabel('Dark Current (e-/s), 5s exposure time')
ax[1].set_xlabel('Dark Current (e-/s), 5s exposure time')
ax[1].set_ylabel('Dark Current (e-/s), 30s exposure time')
ax[2].set_xlabel('Dark Current (e-/s), 30s exposure time')
ax[2].set_ylabel('Dark Current (e-/s), 600s exposure time')
# Add best fit ratio
ax[0].text(0.5, 0, 'Best Fit Ratio: ' + format(popt1[0], '4.3f'), transform=ax[0].transAxes)
ax[1].text(0.5, 0, 'Best Fit Ratio: ' + format(popt2[0], '4.3f'), transform=ax[1].transAxes)
ax[2].text(0.5, 0, 'Best Fit Ratio: ' + format(popt3[0], '4.3f'), transform=ax[2].transAxes)
label_plot(dark_current_stack[4])
plt.show()

In [ ]:
print(fit_points_1s.shape)

Load stacks of dark frames for a given readout mode, at 1us, 1s, 5s, 30s, 120s, and 600s

In [ ]:
# folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Defect Pixels/HS_HG_RS'
# stack = get_stacks(folder)
mean_img_dict = get_mean_images(stack)

Visually inspect mean frames for the 1us and 1s dark exposures, as well as the bias-subtracted 1s exposure

In [ ]:
%matplotlib inline
mode = 'HS_HG_RS'
stack_num_bias = 0
stack_num_1s = 5
stack_num_5s = 4
stack_num_30s = 6
stack_num_120s = 1
stack_num_600s = 3

mean_bias = mean_img_dict[f'{mode}_1us']
mean_1s = mean_img_dict[f'{mode}_1000000us']
mean_5s = mean_img_dict[f'{mode}_5000000us']
mean_30s = mean_img_dict[f'{mode}_30000000us']
mean_120s = mean_img_dict[f'{mode}_120000000us']
# mean_600s = mean_img_dict[f'{mode}_600s']
mean_1s_subtract = mean_1s - mean_bias
mean_5s_subtract = mean_5s - mean_bias
mean_30s_subtract = mean_30s - mean_bias
mean_120s_subtract = mean_120s - mean_bias
# mean_600s_subtract = mean_600s - mean_bias
# Get 1st and 99th percentile values for bias subtraction
bias_low = np.percentile(mean_bias, 1)
bias_high = np.percentile(mean_bias, 99)
bias_subtract_low = np.percentile(mean_1s_subtract, 1)
bias_subtract_high = np.percentile(mean_1s_subtract, 99)
plt.imshow(mean_bias, vmin=bias_low, vmax=bias_high)
label_plot(stack[stack_num_bias])
plt.colorbar()
plt.title('Mean Stack of 50 Dark 1us Bias Exposures')
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.show()

plt.imshow(mean_1s, vmin=bias_low, vmax=bias_high)
label_plot(stack[stack_num_1s])
plt.colorbar()
plt.title('Mean Stack of 50 Dark 1s Exposures')
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.show()

plt.imshow(mean_1s_subtract, vmin=bias_subtract_low, vmax=bias_subtract_high)
label_plot(stack[stack_num_bias])
plt.colorbar()
plt.title('Mean Stack of 50 Dark 1s Exposures, Bias-Subtracted')
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.show()

Plot histograms for the bias-subtracted 1s frame

In [ ]:
plt.imshow(mean_1s_subtract, vmin=bias_subtract_low, vmax=bias_subtract_high)
plt.colorbar()
label_plot(stack[stack_num_1s])
plt.title('Pixel Values: Mean Stack of 1s Exposures, Bias-Subtracted')
# Put a box around top left corner
plt.gca().add_patch(plt.Rectangle((0, 0), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((6496, 0), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((0, 6496), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((6496, 6496), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((3248, 3248), 1624, 1624, fill=False, edgecolor='red'))
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.show()

# bias_subtract_low = -10
# bias_subtract_high = 10
plt.hist(mean_1s_subtract.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step')
plt.axvline(0, color='black', linestyle='dashed')
plt.xlabel('Pixel Value Difference')
plt.ylabel('Number of Pixels')
plt.legend()
label_plot(stack[stack_num_1s])
plt.title('Mean Stack of 50 Dark 1s Exposures, Bias-Subtracted')
plt.show()

mean_1s_subtract_top_left = mean_1s_subtract[:1624, :1624]
mean_1s_subtract_top_right = mean_1s_subtract[:1624, 6496:]
mean_1s_subtract_bottom_left = mean_1s_subtract[6496:, :1624]
mean_1s_subtract_bottom_right = mean_1s_subtract[6496:, 6496:]
mean_1s_subtract_center = mean_1s_subtract[3248:4872, 3248:4872]
plt.hist(mean_1s_subtract_top_left.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Top Left')
plt.hist(mean_1s_subtract_top_right.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Top Right')
plt.hist(mean_1s_subtract_bottom_left.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Bottom Left')
plt.hist(mean_1s_subtract_bottom_right.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Bottom Right')
plt.hist(mean_1s_subtract_center.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Center')
plt.axvline(0, color='black', linestyle='dashed')
plt.xlabel('Pixel Value Difference')
plt.ylabel('Number of Pixels')
label_plot(stack[stack_num_1s])
plt.legend()
plt.title('Pixel Values: Mean Stack of 1s Exposures, Bias-Subtracted')
plt.show()

Check for any row-to-row variations, which can be indicative of different bias levels on different readout channels.

In [ ]:
# Plot just the center 10 rows and center 50 columns in the bias frame
plt.imshow(mean_bias[4060:4090, 4060:4090], vmin=360, vmax=380)
plt.xlabel('Column Number (relative to sensor center)')
plt.ylabel('Row Number (relative to sensor center)')
plt.colorbar()
label_plot(stack[stack_num_bias])
plt.title('Zoomed-In View of Bias Frame Center')
plt.show()

even_rows_bias = np.mean(mean_bias[::2], axis=1)
odd_rows_bias = np.mean(mean_bias[1::2], axis=1)
even_cols_bias = np.mean(mean_bias[:, ::2], axis=0)
odd_cols_bias = np.mean(mean_bias[:, 1::2], axis=0)
plt.plot(np.arange(0, 8120, 2), even_rows_bias, label='Even Rows')
plt.plot(np.arange(1, 8120, 2), odd_rows_bias, label='Odd Rows')
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value in Row')
plt.ylim(bias_low, bias_high)
label_plot(stack[stack_num_bias])
plt.legend()
plt.title('Bias Frame Pixel Values Across Rows')
plt.show()

plt.plot(np.arange(0, 8120, 2), even_cols_bias, label='Even Columns')
plt.plot(np.arange(1, 8120, 2), odd_cols_bias, label='Odd Columns')
plt.xlabel('Column Number')
plt.ylabel('Mean Pixel Value in Column')
plt.ylim(bias_low, bias_high)
label_plot(stack[stack_num_bias])
plt.legend()
plt.title('Bias Frame Pixel Values Across Columns')
plt.show()


Look at alternating row behavior for different exposure times. This yields an extra slide for modes where alternating-row behavior has been identified.

In [ ]:
rows_mod_4_0 = np.mean(mean_bias[0::4], axis=1)
rows_mod_4_1 = np.mean(mean_bias[1::4], axis=1)
rows_mod_4_2 = np.mean(mean_bias[2::4], axis=1)
rows_mod_4_3 = np.mean(mean_bias[3::4], axis=1)
plt.plot(np.arange(0, 8120, 4), rows_mod_4_0, label='Row # Mod 4 = 0')
plt.plot(np.arange(1, 8120, 4), rows_mod_4_1, label='Row # Mod 4 = 1')
plt.plot(np.arange(2, 8120, 4), rows_mod_4_2, label='Row # Mod 4 = 2')
plt.plot(np.arange(3, 8120, 4), rows_mod_4_3, label='Row # Mod 4 = 3')
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value in Row')
plt.ylim(bias_low, bias_high)
label_plot(stack[stack_num_bias])
plt.legend()
plt.title('Bias Frame Pixel Values Across Rows')
plt.show()

even_rows_1s = np.mean(mean_1s[::2], axis=1)
odd_rows_1s = np.mean(mean_1s[1::2], axis=1)
plt.plot(np.arange(0, 8120, 2), even_rows_bias, label='Bias Frame, Even Rows')
plt.plot(np.arange(0, 8120, 2), odd_rows_bias, label='Bias Frame, Odd Rows')
plt.plot(np.arange(0, 8120, 2), even_rows_1s, label='1s Frame, Even Rows')
plt.plot(np.arange(0, 8120, 2), odd_rows_1s, label='1s Frame, Odd Rows')
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value in Row')
plt.ylim(bias_low, bias_high)
label_plot(stack[stack_num_1s])
plt.legend()
plt.title('Bias Frame and 1s Frame Pixel Values Across Rows')
plt.show()

even_rows_5s = np.mean(mean_5s[::2], axis=1)
odd_rows_5s = np.mean(mean_5s[1::2], axis=1)
even_rows_30s = np.mean(mean_30s[::2], axis=1)
odd_rows_30s = np.mean(mean_30s[1::2], axis=1)
even_rows_120s = np.mean(mean_120s[::2], axis=1)
odd_rows_120s = np.mean(mean_120s[1::2], axis=1)
color_blue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
color_orange = plt.rcParams['axes.prop_cycle'].by_key()['color'][1]
plt.plot(np.arange(0, 8120, 2), even_rows_bias, label='Even Rows', color=color_blue)
plt.plot(np.arange(0, 8120, 2), odd_rows_bias, label='Odd Rows', color=color_orange)
plt.plot(np.arange(0, 8120, 2), even_rows_1s + 5, color=color_blue)
plt.plot(np.arange(0, 8120, 2), odd_rows_1s + 5, color=color_orange)
plt.plot(np.arange(0, 8120, 2), even_rows_5s + 10, color=color_blue)
plt.plot(np.arange(0, 8120, 2), odd_rows_5s + 10, color=color_orange)
plt.plot(np.arange(0, 8120, 2), even_rows_30s + 15, color=color_blue)
plt.plot(np.arange(0, 8120, 2), odd_rows_30s + 15, color=color_orange)
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value in Row + Offset')
plt.ylim(195, 220)
plt.xlim(0, 10000)
label_plot(stack[stack_num_bias])
# Put label for bias as text on right
plt.text(8200, 200, 'Bias Frame')
plt.text(8200, 205, '1s Frame')
plt.text(8200, 210, '5s Frame')
plt.text(8200, 218, '30s Frame')
plt.title('Alternating Row Behavior at Different Exposure Times')
plt.legend()
plt.show()


Look at individual images. This yields an extra slide for modes where alternating-row behavior has been identified.

In [ ]:
bias_0 = stack[stack_num_bias]['imagestack'][0].astype(int)
bias_last = stack[stack_num_bias]['imagestack'][-1].astype(int)
even_rows_bias_0 = np.mean(bias_0[::2], axis=1)
odd_rows_bias_0 = np.mean(bias_0[1::2], axis=1)
even_rows_bias_last = np.mean(bias_last[::2], axis=1)
odd_rows_bias_last = np.mean(bias_last[1::2], axis=1)
plt.plot(np.arange(0, 8120, 2), even_rows_bias_0, label='Even Rows, First Frame')
plt.plot(np.arange(0, 8120, 2), odd_rows_bias_0, label='Odd Rows, First Frame')
plt.plot(np.arange(0, 8120, 2), even_rows_bias_last + 50, label='Even Rows, Last Frame')
plt.plot(np.arange(0, 8120, 2), odd_rows_bias_last + 50, label='Odd Rows, Last Frame')
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value in Row + Offset')
plt.ylim(bias_low, bias_high)
label_plot(stack[stack_num_bias])
plt.legend()
plt.title('Bias Frame Pixel Values Across Rows, First and Last Frames')
plt.show()

# See if even/odd difference changes based on day of acquisition
mean_bias_2 = np.mean(stack[2]['imagestack'], axis=0)
even_rows_bias_2 = np.mean(mean_bias_2[::2], axis=1)
odd_rows_bias_2 = np.mean(mean_bias_2[1::2], axis=1)
plt.plot(np.arange(0, 8120, 2), even_rows_bias, label='Even Rows, Stack Collected 4/23')
plt.plot(np.arange(0, 8120, 2), odd_rows_bias, label='Odd Rows, Stack Collected 4/23')
plt.plot(np.arange(0, 8120, 2), even_rows_bias_2 + 10, label='Even Rows, Stack Collected 4/18')
plt.plot(np.arange(0, 8120, 2), odd_rows_bias_2 + 10, label='Odd Rows, Stack Collected 4/18')
plt.xlabel('Row Number')
plt.ylabel('Mean Pixel Value in Row + Offset')
plt.ylim(bias_low, bias_high + 10)
label_plot(stack[stack_num_bias])
plt.legend()
plt.title('Bias Frame Pixel Values Across Rows, Different Collection Dates')
plt.show()

Visually inspect bias-subtracted dark frames at longer exposure times. Plot pixel value histograms for these exposure times and for different parts of the sensor for 120s exposure.

In [ ]:
# bias_subtract_low = -10
# bias_subtract_high = 50
plt.hist(mean_1s_subtract.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='1s Bias Subtracted')
plt.hist(mean_30s_subtract.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='30s Bias Subtracted')
plt.hist(mean_120s_subtract.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='120s Bias Subtracted')
plt.hist(mean_600s_subtract.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='600s Bias Subtracted')
plt.axvline(0, color='black', linestyle='dashed')
plt.xlabel('Pixel Value Difference')
plt.ylabel('Number of Pixels')
plt.legend()
label_plot(stack[stack_num_bias])
plt.title('Pixel Values: Bias-Subtracted Dark Images at Varying Exposure Times')
plt.show()

plt.imshow(mean_120s_subtract, vmin=bias_subtract_low, vmax=bias_subtract_high)
plt.gca().add_patch(plt.Rectangle((0, 0), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((6496, 0), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((0, 6496), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((6496, 6496), 1624, 1624, fill=False, edgecolor='red'))
plt.gca().add_patch(plt.Rectangle((3248, 3248), 1624, 1624, fill=False, edgecolor='red'))
plt.colorbar()
label_plot(stack[stack_num_120s])
plt.title('Mean Stack of 50 Dark 120s Exposures, Bias-Subtracted')
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.show()

mean_120s_subtract_top_left = mean_120s_subtract[:1624, :1624]
mean_120s_subtract_top_right = mean_120s_subtract[:1624, 6496:]
mean_120s_subtract_bottom_left = mean_120s_subtract[6496:, :1624]
mean_120s_subtract_bottom_right = mean_120s_subtract[6496:, 6496:]
mean_120s_subtract_center = mean_120s_subtract[3248:4872, 3248:4872]
plt.hist(mean_120s_subtract_top_left.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Top Left')
plt.hist(mean_120s_subtract_top_right.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Top Right')
plt.hist(mean_120s_subtract_bottom_left.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Bottom Left')
plt.hist(mean_120s_subtract_bottom_right.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Bottom Right')
plt.hist(mean_120s_subtract_center.flatten(), bins=100, range=(bias_subtract_low, bias_subtract_high), histtype='step', label='Center')
plt.axvline(0, color='black', linestyle='dashed')
plt.xlabel('Pixel Value Difference')
plt.ylabel('Number of Pixels')
plt.legend()
label_plot(stack[stack_num_120s])
plt.title('Pixel Values: Mean Stack of 120s Exposures, Bias-Subtracted')
plt.show()

In [ ]:
mode1 = mode.replace('_', '')
if mode1 != 'HDRRS':
    gain = cosmos_gain_dict[f'XRgain_{mode1}']
else:
    gain = 0.4
np.random.seed(2)
num_pix = 10
pixel_indices = np.random.randint(0, 8120, (num_pix, 2))
for i in range(num_pix):
    row, col = pixel_indices[i]
    offset = i * 3
    plt.plot([0, 1, 5, 30, 120, 600], [mean_bias[row, col] + offset, mean_1s[row, col] + offset, mean_5s[row, col] + offset, mean_30s[row, col] + offset, mean_120s[row, col] + offset, mean_600s[row, col] + offset], marker='o', label=f'Pixel {i}')
plt.xlabel('Exposure Time (s)')
plt.ylabel('Pixel Value + Offset')
label_plot(stack[stack_num_bias])
plt.title('Pixel Value vs. Exposure Time for 10 Random Pixels')
plt.show()

dc_frame_120s = (mean_120s - mean_1s) * gain / 119
dc_frame_600s = (mean_600s - mean_1s) * gain / 599
def func(x, a):
    return a * x
fit_points_30s = dc_frame_30s[(dc_frame_30s < 10) & (dc_frame_600s < 10) & (dc_frame_120s < 10)]
fit_points_120s = dc_frame_120s[(dc_frame_120s < 10) & (dc_frame_600s < 10) & (dc_frame_30s < 10)]
fit_points_600s = dc_frame_600s[(dc_frame_120s < 10) & (dc_frame_600s < 10) & (dc_frame_30s < 10)]
popt, pcov = curve_fit(func, fit_points_120s.flatten(), fit_points_600s.flatten())
x_points = np.linspace(0, 20, 100)
plt.plot(x_points, popt[0] * x_points, color='red', label='Best Fit Line')
plt.plot(x_points, x_points, color='black', linestyle='dashed', label='Expected Relationship')
hist1 = plt.hist2d(dc_frame_120s[:, :].flatten(), dc_frame_600s[:, :].flatten(), bins=100, range=[[-0.5, 5], [-0.5, 5]], norm='log')
# Label the colorbar
plt.colorbar(label='Number of Pixels')
plt.xlabel('Dark Current (e-/s), 120s exposure time')
plt.ylabel('Dark Current (e-/s), 600s exposure time')
# Add best fit ratio
# plt.text(3.0, -0.25, 'Ideal Relationship')
# Center the text
plt.text(2.5, -2, 'Best Fit Ratio: ' + format(popt[0], '4.3f'), color='red', ha='center')
label_plot(stack[stack_num_600s])
plt.title('Dark Current Relationship Between 120s and 600s Exposures')
plt.show()


In [ ]:
dark_current_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/Dark Current/HS_HG_RS'
dark_current_stack = get_stack(dark_current_folder)
dark_current_dict = get_mean_images(dark_current_stack)

In [ ]:
mode = 'HS_HG_RS'
dark_frame_1us = dark_current_dict[f'{mode}_1us']
# dark_frame_1s = dark_current_dict[f'{mode}_1s']
dark_frame_5s = dark_current_dict[f'{mode}_5s']
dark_frame_30s = dark_current_dict[f'{mode}_30s']
dark_frame_600s = dark_current_dict[f'{mode}_600s']

In [ ]:
np.random.seed(2)
num_pix = 5
pixel_indices = np.random.randint(0, 8120, (num_pix, 2))
for i in range(num_pix):
    row, col = pixel_indices[i]
    offset = i * 300
    plt.plot([0, 1, 5, 30, 600], [dark_frame_1us[row, col] + offset, dark_frame_1s[row, col] + offset, dark_frame_5s[row, col] + offset, dark_frame_30s[row, col] + offset, dark_frame_600s[row, col] + offset], marker='o', label=f'Pixel {i}')
plt.xlabel('Exposure Time (s)')
plt.ylabel('Pixel Value + Offset')
label_plot(dark_current_stack[0])
plt.title('Pixel Value vs. Exposure Time for 5 Random Pixels')
plt.show()

dc_frame_30s = (dark_frame_30s - dark_frame_5s) * cosmos_gain_dict['XRgain_HSHGGS'] / 25
dc_frame_600s = (dark_frame_600s - dark_frame_5s) * cosmos_gain_dict['XRgain_HSHGGS'] / 595
def func(x, a):
    return a * x
fit_points_30s = dc_frame_30s[(dc_frame_30s < 5) & (dc_frame_600s < 5)]
fit_points_600s = dc_frame_600s[(dc_frame_30s < 5) & (dc_frame_600s < 5)]
popt, pcov = curve_fit(func, fit_points_30s.flatten(), fit_points_600s.flatten())
x_points = np.linspace(0, 20, 100)
plt.plot(x_points, popt[0] * x_points, color='red', label='Best Fit Line')
plt.plot(x_points, x_points, color='black', linestyle='dashed', label='Expected Relationship')
hist1 = plt.hist2d(dc_frame_30s[:, :].flatten(), dc_frame_600s[:, :].flatten(), bins=100, range=[[-0.5, 20], [-0.5, 20]], norm='log')
# Label the colorbar
plt.colorbar(label='Number of Pixels')
plt.xlabel('Dark Current (e-/s), 30s exposure time')
plt.ylabel('Dark Current (e-/s), 600s exposure time')
# Add best fit ratio
plt.text(10.0, 1.0, 'Ideal Relationship')
plt.text(10.0, 0, 'Best Fit Ratio: ' + format(popt[0], '4.3f'), color='red')
label_plot(dark_current_stack[0])
plt.show()

In [ ]:
plt.imshow(dark_frame_1s, vmin=np.percentile(dark_frame_1s, 5), vmax=np.percentile(dark_frame_1s, 95))
label_plot(dark_current_stack[0])
plt.colorbar()
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.title('Mean Stack of Dark 1s Exposures')
plt.show()
plt.imshow(dc_frame_30s, vmin=np.percentile(dc_frame_30s, 5), vmax=np.percentile(dc_frame_30s, 95))
label_plot(dark_current_stack[3])
plt.colorbar(label='Dark Current (e-/s)')
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.title('Dark Current in 30s Exposure')
plt.show()
plt.imshow(dc_frame_600s, vmin=np.percentile(dc_frame_600s, 5), vmax=np.percentile(dc_frame_600s, 95))
label_plot(dark_current_stack[2])
plt.colorbar(label='Dark Current (e-/s)')
plt.xlabel('Column Number')
plt.ylabel('Row Number')
plt.title('Dark Current in 600s Exposure')
plt.show()